# Guide for simulating with AlphaPEM

## Purpose
This notebook shows two ways to run AlphaPEM simulations:

1) The Python API `run_alphapem_from_df` for small interactive runs
2) The parallel CLI script `scripts/run_sampler_batch_parallel.py` for larger runs

You will:
- Run three configurations and see how errors are handled
- Create a Latin LHS design with 20 samples and save it in `data/designs`
- Launch a parallel batch for 10 configurations and inspect the outputs

> Tip: Run this notebook from the repository root inside `notebooks/` so the relative paths match!

In [1]:
from pathlib import Path
import os, sys, json
import numpy as np
import pandas as pd

# Assume this notebook lives in notebooks/
REPO_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
print("Repo root resolved to:", REPO_ROOT)

# Make src importable
if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))

from src.sampling.sampler import run_alphapem_from_df
from src.sampling.bounds import load_param_config

# Project paths
alpha_pem_root = str(REPO_ROOT / "external" / "AlphaPEM")
param_config_yaml = str(REPO_ROOT / "configs" / "param_config.yaml")
simulator_defaults_yaml = str(REPO_ROOT / "configs" / "simulator_defaults.yaml")
raw_dir = REPO_ROOT / "data" / "raw"
designs_dir = REPO_ROOT / "data" / "designs"
raw_dir.mkdir(parents=True, exist_ok=True)
designs_dir.mkdir(parents=True, exist_ok=True)

alpha_pem_root, param_config_yaml, simulator_defaults_yaml

Repo root resolved to: c:\Users\User\Documents\0. Semestres\SS2025\Official-Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells


('c:\\Users\\User\\Documents\\0. Semestres\\SS2025\\Official-Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\\external\\AlphaPEM',
 'c:\\Users\\User\\Documents\\0. Semestres\\SS2025\\Official-Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\\configs\\param_config.yaml',
 'c:\\Users\\User\\Documents\\0. Semestres\\SS2025\\Official-Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\\configs\\simulator_defaults.yaml')

## Prerequisites

- The AlphaPEM code is present in `external/AlphaPEM` and importable.
- Your YAML files exist at `configs/param_config.yaml` and `configs/simulator_defaults.yaml`

If you use a virtual environment, activate it before starting the Jupyter server

## Controlling the simulation conditions and parameter ranges

The file `configs/param_config.yaml` is the single source of truth for how we define inputs, ranges, types, fixed values, and derived relationships . Every sampler and simulator call in this project uses it to load and validate the parameter space before running AlphaPEM. Concretely:

* `load_param_config(path)` parses `param_config.yaml` into a `SamplingSpec` that contains names, types, numeric bounds for continuous and integer parameters, allowed categories for categoricals, fixed values and derived expressions.
* `validate_sample_against_spec(sample, spec)` checks that a sample adheres to bounds and types, enforces fixed parameters and first applies any derived expressions declared in the YAML. If something is wrong it raises a `SampleValidationError`.
* Example of a derived expression: `Pc_des` can be derived as `Pa_des - 20000`. This gets computed automatically during validation as long as your YAML declares it as fixed with a `derived` expression.

### What must be in `param_config.yaml`

The YAML must list the 19 AlphaPEM input features we use in this project:

1. The 7 operating conditions
   `Tfc, Pa_des, Pc_des, Sa, Sc, Phi_a_des, Phi_c_des`

2. The undetermined physical parameters used for sensitivity analysis
   `epsilon_gdl, tau, epsilon_mc, epsilon_c, e, Re, i0_c_ref, kappa_co, kappa_c`

3. Project‑specific fixed physical constants we keep fixed in runs
   `a_slim, b_slim, a_switch`  (we keep them fixed here so the space focuses on the undetermineds)

For any feature whose sensitivity you want to explore, provide:

* `type` one of `continuous`, `integer`, or `categorical`
* a numeric range `low` and `high` for continuous or integer types
* or `values` for categorical types

You can also fix parameters directly in the YAML:

* Example: fix `Sa` to 1.3 across all samples
* Example: derive `Pc_des` from `Pa_des` using `derived: "Pa_des - 20000"`

A minimal YAML skeleton looks like this:

```yaml
parameters:
  # -------- Operating conditions --------
  - name: Tfc
    type: continuous
    low: 333
    high: 363
    fixed: false

  - name: Pc_des
    type: continuous
    fixed: true
    derived: "Pa_des - 20000"   # derived rule applied by bounds.py

  - name: Sa
    type: continuous
    fixed: true
    value: 1.3                   # example of fixing Sa for all runs
```

The samplers and runners fill missing fixed values, compute derived values, and enforce bounds using this spec before any simulation starts. This keeps everything reproducible across notebooks and scripts.

In [3]:
from src.sampling.bounds import load_param_config, validate_sample_against_spec, SampleValidationError

spec = load_param_config(param_config_yaml)

# Build a readable table of the parameter space
rows = []
for name, p in spec.spec_index.items():
    row = {
        "name": name,
        "type": p["type"],
        "fixed": bool(p.get("fixed", False)),
        "low": p.get("low", None),
        "high": p.get("high", None),
        "values": p.get("values", None),
        "value_if_fixed": p.get("value", None),
        "derived_expr": p.get("derived", None),
    }
    rows.append(row)

spec_df = pd.DataFrame(rows).sort_values(["fixed","name"], ascending=[True, True]).reset_index(drop=True)
display(spec_df)

print(f"Variables counted as non-fixed: {len(spec.names)}")
print(f"Bounds array shape: {spec.bounds.shape}")

,name,type,fixed,low,high,values,value_if_fixed,derived_expr
0,Pa_des,continuous,False,1.300000e+05,300000.000000,None,NaN,None
1,Phi_c_des,continuous,False,1.000000e-01,0.700000,None,NaN,None
2,Re,continuous,False,5.000000e-07,0.000005,None,NaN,None
3,Sc,continuous,False,1.100000e+00,3.000000,None,NaN,None
4,Tfc,continuous,False,3.330000e+02,363.000000,None,NaN,None
5,e,integer,False,3.000000e+00,5.000000,None,NaN,None
6,epsilon_c,continuous,False,1.500000e-01,0.300000,None,NaN,None
7,epsilon_gdl,continuous,False,5.500000e-01,0.800000,None,NaN,None
8,epsilon_mc,continuous,False,1.500000e-01,0.400000,None,NaN,None
9,i0_c_ref,continuous,False,1.000000e-03,80.000000,None,NaN,None


Variables counted as non-fixed: 13
Bounds array shape: (13, 2)


Let's validate a sample that omits Pc_des but includes Pa_des.
Note that Pc_des will be computed from Pa_des if the YAML declares 'Pc_des' as fixed with derived: 'Pa_des - 20000'.

In [5]:

good_sample = {
    "Tfc": 340.0,
    "Pa_des": 230000.0,
    "Sa": 1.3,               # fixed in YAML in this example
    "Sc": 2.0,
    "Phi_a_des": 0.5,
    "Phi_c_des": 0.4,
    "epsilon_gdl": 0.62,
    "tau": 2.0,
    "epsilon_mc": 0.25,
    "epsilon_c": 0.22,
    "e": 4,                  # integer
    "Re": 1.0e-6,
    "i0_c_ref": 10.0,
    "kappa_co": 25.0,
    "kappa_c": 1.0,
    # Pc_des not provided on purpose, to be derived by validator
}

aug = validate_sample_against_spec(good_sample, spec)
print("\nAugmented sample after validation and derived application:")
for k in sorted(aug.keys()):
    print(f"  {k}: {aug[k]}")


Augmented sample after validation and derived application:
  Pa_des: 230000.0
  Pc_des: 210000.0
  Phi_a_des: 0.5
  Phi_c_des: 0.4
  Re: 1e-06
  Sa: 1.3
  Sc: 2.0
  Tfc: 340.0
  a_slim: 0
  a_switch: 0.99
  b_slim: 1
  e: 4
  epsilon_c: 0.22
  epsilon_gdl: 0.62
  epsilon_mc: 0.25
  i0_c_ref: 10.0
  kappa_c: 1.0
  kappa_co: 25.0
  tau: 2.0


Let's now provoke a validation error. For that, we set Tfc out of bounds or set e to a non-integer.

In [7]:
# 
bad_sample = dict(good_sample)
bad_sample["Tfc"] = 1000.0     # out of bounds on purpose

try:
    validate_sample_against_spec(bad_sample, spec)
except SampleValidationError as e:
    print("\nExpected validation error captured:")
    print(" ", e)

bad_sample = dict(good_sample)
bad_sample["e"] = 5.6   
try:
    validate_sample_against_spec(bad_sample, spec)
except SampleValidationError as e:
    print("\nExpected validation error captured:")
    print(" ", e)


Expected validation error captured:
  Tfc out of bounds [333.0, 363.0]: 1000.0

Expected validation error captured:
  e must be integer in [3, 5], got 5.6


## Minimal demo with three configurations

We will run three rows:
- One good configuration inside the bounds.
- Two faulty configurations with `Tfc` outside the allowed range to trigger validation errors.

We turn on `verify=True`, which enforces bounds and computes derived parameters strictly from the YAML (for example `Pc_des` from `Pa_des` if defined).

In [ ]:
df_configs = pd.DataFrame([
    {
        "config_id": "good_cfg",
        "Tfc": 345.0,
        "Pa_des": 220000.0,
        "Sc": 2.2,
        "Phi_c_des": 0.3,
        "epsilon_gdl": 0.65,
        "tau": 2.5,
        "epsilon_mc": 0.25,
        "epsilon_c": 0.22,
        "e": 4,
        "Re": 1.2e-6,
        "i0_c_ref": 10.0,
        "kappa_co": 22.0,
        "kappa_c": 1.0
    },
    {
        "config_id": "bad_cfg_1",
        "Tfc": 300.0,  # out of bounds to show error
        "Pa_des": 200000.0,
        "Sc": 2.0,
        "Phi_c_des": 0.5,
        "epsilon_gdl": 0.62,
        "tau": 2.0,
        "epsilon_mc": 0.30,
        "epsilon_c": 0.28,
        "e": 5,
        "Re": 1.0e-6,
        "i0_c_ref": 20.0,
        "kappa_co": 30.0,
        "kappa_c": 0.9
    },
    {
        "config_id": "bad_cfg_2",
        "Tfc": 600.0,  # out of bounds to show error
        "Pa_des": 260000.0,
        "Sc": 1.8,
        "Phi_c_des": 0.4,
        "epsilon_gdl": 0.60,
        "tau": 1.5,
        "epsilon_mc": 0.20,
        "epsilon_c": 0.18,
        "e": 3,
        "Re": 5.0e-7,
        "i0_c_ref": 5.0,
        "kappa_co": 18.0,
        "kappa_c": 0.7
    },
])

df_configs.insert(0, "index", range(len(df_configs)))
df_configs

In [ ]:
results_df = run_alphapem_from_df(
    df_configs,
    alpha_pem_root=alpha_pem_root,
    simulator_defaults_yaml=simulator_defaults_yaml,
    param_config_yaml=param_config_yaml,
    verify=True,
    run_name="notebook_three_configs",
    output_dir=str(raw_dir),
    results_format="pkl",
    save_every=2,
    print_errors=True,
)

print("Rows:", len(results_df))
results_df.head(10)

After the run you will have:
- `data/raw/notebook_three_configs_simulations.pkl` with the results table. It includes the raw `ifc` and `Ucell` arrays plus the expanded `ifc_1..ifc_31` and `Ucell_1..Ucell_31` columns.
- `data/raw/notebook_three_configs_sim_errors.csv` listing any rows that failed validation or raised runtime errors.

You should see two errors for the two faulty rows and one successful simulation.

In [ ]:
# Inspect the logged errors
err_path = raw_dir / "notebook_three_configs_sim_errors.csv"
if err_path.exists():
    display(pd.read_csv(err_path))
else:
    print("No errors file found at", err_path)

## Create a Latin LHS design with 20 samples

We will load the parameter spec from `param_config.yaml`, generate a Latin Hypercube sample for the non-fixed parameters, round integer parameters, and then save the design to both PKL and CSV under `data/designs/`

In [ ]:
# Try to use SciPy's LHS if available, otherwise fallback
try:
    from scipy.stats import qmc  # type: ignore
    has_scipy = True
except Exception:
    has_scipy = False

spec = load_param_config(param_config_yaml)
var_names = spec.names
bounds = spec.bounds
types = spec.types
d = len(var_names)
n_samples = 20

if has_scipy:
    sampler = qmc.LatinHypercube(d=d, seed=123)
    lhs_unit = sampler.random(n=n_samples)
else:
    # Simple LHS fallback
    rng = np.random.default_rng(123)
    lhs_unit = np.zeros((n_samples, d))
    for j in range(d):
        perm = rng.permutation(n_samples)
        lhs_unit[:, j] = (perm + rng.random(n_samples)) / n_samples

# Scale to bounds
samples = bounds[:, 0] + lhs_unit * (bounds[:, 1] - bounds[:, 0])

# Round integer parameters
for j, t in enumerate(types):
    if t == "integer":
        samples[:, j] = np.round(samples[:, j]).astype(int)

df_design = pd.DataFrame(samples, columns=var_names)
df_design.insert(0, "index", range(len(df_design)))
df_design.insert(1, "config_id", [f"lhs20_{i:03d}" for i in range(len(df_design))])

design_base = "lhs20_demo"
pkl_path = designs_dir / f"{design_base}.pkl"
csv_path = designs_dir / f"{design_base}.csv"
df_design.to_pickle(pkl_path)
df_design.to_csv(csv_path, index=False)

pkl_path, csv_path, df_design.head()

## Run the parallel batch script for 10 configs

We will call `scripts/run_sampler_batch_parallel.py` from inside the notebook using `subprocess`. We take the 20-sample design we just saved and run only the *last* 10 rows with multiple workers.

This script:
- splits the DataFrame into worker chunks
- validates rows if `--verify` is used
- writes per-worker checkpoints under `data/raw/temp/<run_name>/`
- merges final results and errors
- writes a metadata JSON with run details

Note: If your environment does not have AlphaPEM or the YAML files, this cell will fail. That is expected outside the full project.

In [ ]:
import subprocess, shlex

run_name = "notebook_parallel_demo"
cmd = f"""
python {shlex.quote(str(REPO_ROOT / 'scripts' / 'run_sampler_batch_parallel.py'))} \
  --input {shlex.quote(str(pkl_path))} \
  --n_samples 10 \
  --offset 10 \
  --alpha_pem_root {shlex.quote(alpha_pem_root)} \
  --param_config_yaml {shlex.quote(param_config_yaml)} \
  --simulator_defaults_yaml {shlex.quote(simulator_defaults_yaml)} \
  --verify \
  --n_workers 4 \
  --save_every 5 \
  --output_dir {shlex.quote(str(raw_dir))} \
  --run_name {shlex.quote(run_name)} \
  --format csv \
  --print_errors
"""

print("Running command:")
print(cmd)

# Uncomment to actually run. It is commented to avoid accidental long runs in unknown environments
# result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)

After the parallel run finishes you will find:
- `data/raw/notebook_parallel_demo_simulations.csv`
- `data/raw/notebook_parallel_demo_sim_errors.csv`
- `data/raw/notebook_parallel_demo_meta.json`

You can read the metadata to quickly review the run.

In [ ]:
meta_path = raw_dir / f"{run_name}_meta.json"
if meta_path.exists():
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    fields = ["status","n_subset","n_workers","ok","err","start_time","end_time","duration_seconds","results_path","errors_path","temp_dir"]
    {k: meta.get(k) for k in fields}
else:
    print("Meta not found at", meta_path)

## Wrap up

We've shown how to:
- Use the Python API for quick iteration and debugging in a notebook
- Use the script for larger runs from designs in `data/designs`
- Keep the YAMLs under version control so runs are reproducible

If something fails, check the errors CSV and the metadata JSON for details. The temp folder under `data/raw/temp/<run_name>` is only kept when a run fails or is interrupted.